# LSTM Training

In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [2]:
import pandas as pd
import numpy as np
import itertools
import os
import random

In [3]:
# read finished Data; case training
def readData(src, noConfFlag=True):
    dataseries = []
    for i in itertools.count():
        try:
            #print(i)
            dataseries.append(np.array(pd.read_json(src+ '/'+os.path.basename(src[:-4])+'_'+str(i).zfill(12)+'_keypoints.json').iloc[0,0]['pose_keypoints_2d']))
            if noConfFlag:
                dataseries[-1] = np.append(dataseries[-1][:2],np.delete(dataseries[-1][2:], slice(None, None, 3)))
        except ValueError:
            break
        except IndexError:
            if noConfFlag:
                dataseries.append(np.zeros(25*2))
            else:
                dataseries.append(np.zeros(25*3))
    return dataseries,i

In [4]:
labelNames=['rennen','gehen']
dataPaths=['data/output/rennen/','data/output/gehen/']
samplePaths=[]
data=[]
labels=[]
#collect paths to data
dirs0 = os.listdir(dataPaths[0])
for p in dirs0:
    samplePaths.append(dataPaths[0]+p+'0')
dirs1 = os.listdir(dataPaths[1])
for p in dirs1:
    samplePaths.append(dataPaths[1]+p+'1')

#shuffel data
random.shuffle(samplePaths)
#print(samplePaths)
minLength = 75
for smp in samplePaths:
    l = smp[-1:]
    smp = smp[:-1]
    td, i = readData(smp)
    #if i<minLength:
    #    minLength = i
    data.append(td[10:60])
    if l == '0':
        labels.append(np.array([1,0]))
    elif l == '1':
        labels.append(np.array([0,1]))
    else:
        #labels.append(np.array([0,0]))
        print("can't load: ",smp)
    print('.', end="")
        
print("data loaded.")
print("min Length: ", minLength)

.............................................data loaded.
min Length:  75


In [5]:
data

[[array([ 43.5748 , 172.706  ,  17.4589 , 204.038  ,   3.74265, 201.45   ,
           0.     ,   0.     ,   0.     ,   0.     ,  44.8752 , 204.105  ,
          57.9229 , 253.617  ,  94.4453 , 277.098  ,  24.0039 , 294.119  ,
          10.9316 , 295.408  ,  39.6831 , 354.116  ,  68.3301 , 427.217  ,
          36.9888 , 292.807  ,  25.2508 , 351.524  ,   0.     ,   0.     ,
          35.7447 , 167.502  ,  44.8721 , 166.206  ,  14.9121 , 171.395  ,
           0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
           0.     ,   0.     ,  91.8346 , 431.115  ,  85.3385 , 433.761  ,
          67.0462 , 435.027  ]),
  array([ 47.4476 , 172.748  ,  25.3166 , 204.117  ,   3.73002, 204.053  ,
           3.73905, 258.864  ,   3.72929, 305.834  ,  47.5474 , 205.349  ,
          64.4266 , 253.654  ,  97.0867 , 275.822  ,  27.9518 , 296.74   ,
          16.2264 , 297.977  ,  50.0789 , 360.608  ,  86.5893 , 428.493  ,
          40.9696 , 296.726  ,  26.596  , 354.128  ,   0.     ,   0

In [6]:
#define constants

time_steps=50
#hidden LSTM units
num_units=128
# 25*3=75; 25*2=50
n_input=50
#learning rate for adam
learning_rate=0.001

n_classes=2
#size of batch
batch_size=1

In [7]:
#shape for LSTM
d = np.concatenate(data, axis=0).reshape((-1,time_steps*50))
l = np.concatenate(labels, axis=0).reshape((-1,2))

In [8]:
print(d.shape,l.shape)
print(d)
print(l)

(45, 2500) (45, 2)
[[ 43.5748  172.706    17.4589  ... 445.502   367.157   441.574  ]
 [  0.        0.        0.      ... 385.541   420.788   389.405  ]
 [569.415   232.796   608.608   ...   0.      275.873   461.146  ]
 ...
 [  3.09406 277.148     3.7501  ... 445.598   295.435   442.959  ]
 [  0.        0.        0.      ... 420.744   475.603   424.716  ]
 [  0.        0.        0.      ... 416.771   172.808   414.207  ]]
[[0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]]


### TODO:
- Normalize/Standerdize

In [9]:
#split data in train and testset
testSize=10 
test_data = d[:testSize,:]
test_labels = l[:testSize,:]
train_data = d[testSize:,:]
train_labels = l[testSize:,:]

In [10]:
test_data.shape, test_labels.shape

((10, 2500), (10, 2))

In [11]:
train_data.shape, train_labels.shape

((35, 2500), (35, 2))

In [12]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

In [13]:
#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

In [14]:
#defining the network
lstm_layer=tf.nn.rnn_cell.LSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

In [15]:
#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

In [16]:
#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [17]:
#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<=30:
        batch_x=train_data[batch_size*iter:batch_size*(iter+1)]

        batch_y=train_labels[batch_size*iter:batch_size*(iter+1)]
        #print(batch_x)
        batch_x=batch_x.reshape((batch_size,time_steps,n_input))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %5==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1

    #calculating test accuracy
    test_data = test_data[:batch_size].reshape((-1, time_steps, n_input))
    test_label = test_labels[:batch_size]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

For iter  5
Accuracy  1.0
Loss  1.4305105e-06
__________________
For iter  10
Accuracy  0.0
Loss  14.525149
__________________
For iter  15
Accuracy  1.0
Loss  1.1920928e-07
__________________
For iter  20
Accuracy  1.0
Loss  3.70734e-05
__________________
For iter  25
Accuracy  1.0
Loss  1.1920928e-07
__________________
For iter  30
Accuracy  0.0
Loss  11.198364
__________________
Testing Accuracy: 1.0


In [18]:
sess.close()

# Visualisation

In [19]:
show_data = d.reshape(-1,n_input)
show_label = [np.where(r==1)[0][0] for r in l]

In [20]:
#show_label

In [21]:
show_data.shape

(2250, 50)

In [22]:
testing = show_data[1]
testing[1]

172.748

In [23]:
pairs = [
        (1,8),(1,2),(1,5),(2,3),(3,4),   
        (5,6),(6,7),(8,9),(9,10),(10,11),
        (8,12),(12,13),(13,14),(1,0),(0,15), 
        (15,17),(0,16),(16,18),
        (14,19),(19,20),(14,21),(11,22),(22,23),
        (11,24)
        ]

In [24]:
pairs[0][1]

8

In [25]:
import pygame
import sys, traceback

# Initialize Pygame.
pygame.init()
# Set size of pygame window.
screen=pygame.display.set_mode((900,750))
# Create empty pygame surface.
background = pygame.Surface(screen.get_size())
# Fill the background white color.
background.fill((255, 255, 255))
# Convert Surface object to make blitting faster.
background = background.convert()
# Copy background to screen (position (0, 0) is upper left corner).
screen.blit(background, (0,0))
# Create Pygame clock object.
clock = pygame.time.Clock()

mainloop = True
# Desired framerate in frames per second. Try out other values.
FPS = 30
# How many seconds the "game" is played.
playtime = 0.0

count = 0.0

speed = 1

try:
    
    while mainloop:
        # Do not go faster than this framerate.
        milliseconds = clock.tick(FPS*speed)
        playtime += milliseconds / 1000.0

        for event in pygame.event.get():
            # User presses QUIT-button.
            if event.type == pygame.QUIT:
                mainloop = False
            elif event.type == pygame.KEYDOWN:
                # User presses ESCAPE-Key
                if event.key == pygame.K_ESCAPE:
                    mainloop = False

        # Print framerate and playtime in titlebar.
        currentSample=int(count/n_input)%len(show_label)
        text = "FPS: {0:.2f}   Playtime: {1:.2f}  NR: {2:d}  Label: ".format(clock.get_fps(), playtime, currentSample+1)
        text += labelNames[show_label[currentSample]]
        pygame.display.set_caption(text)
        background.fill((255, 255, 255))
        f = show_data[int(count)%(show_data.shape[0])]
        
        for p in pairs:
            x1=int(f[p[0]*2])
            y1=int(f[p[0]*2+1])
            x2=int(f[p[1]*2])
            y2=int(f[p[1]*2+1])
            if not (x1 == 0 or x2 == 0 or y1 == 0 or y2 == 0):
                pygame.draw.line(background, (0, 0, 0), [x1,y1], [x2,y2], 1)
        for i in range(0,f.shape[0],2):#2 for no conf
            #confColor = 255 - int(255*f[i+2])%255
            #pygame.draw.rect(background, (confColor, confColor, confColor), (int(f[i]),int(f[i+1]),4,4)) #bw
            #pygame.draw.rect(background, (confColor, 0, 255-confColor), (int(f[i]),int(f[i+1]),4,4)) #rb
            if i!=25:
                pygame.draw.rect(background, (0, 0, 0), (int(f[i]),int(f[i+1]),4,4)) #no Conf
            else:
                pygame.draw.rect(background, (255, 0, 0), (int(f[i]),int(f[i+1]),4,4)) #no Conf
        #screen.blit(background, (300-f[8*3],300-f[8*3+1]))
        screen.blit(background, (0,0))
        #Update Pygame display.
        pygame.display.update()

        count += 1
except TypeError:
    print("Type error: ", count," ", i)
    traceback.print_exc(file=sys.stdout)
except:
    traceback.print_exc(file=sys.stdout)
else:
    pygame.quit()
# Finish Pygame.
pygame.quit()

# At the very last:
print("This game was played for {0:.2f} seconds".format(playtime))

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
This game was played for 201.05 seconds


# Live Data-read

In [25]:
import sys
import time
import logging
from watchdog.observers import Observer
from watchdog.events import LoggingEventHandler
from watchdog.events import PatternMatchingEventHandler  

In [26]:
class MyHandler(PatternMatchingEventHandler):
    patterns = ["*.json"]
    data = []

    def process(self, event):
        """
        event.event_type 
            'modified' | 'created' | 'moved' | 'deleted'
        event.is_directory
            True | False
        event.src_path
            path/to/observed/file
        """
        # the file will be processed there
        try:
            #print(i)
            f = pd.read_json(event.src_path).iloc[0,0]['pose_keypoints_2d']
            self.data.append(f)
            print(f)
        except ValueError:
            print("ERROR reading: ",event.src_path, event.event_type)


    def on_created(self, event):
        self.process(event)
        
observer = Observer()
observer.schedule(MyHandler(), '.')
observer.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()